In [2]:
import numpy as np
import pandas as pd
import lib
import os

pwd = os.getcwd()
# load ravdess.npy , savee.npy and tess.npy
ravdess_data = np.load('ravdess.npy')
savee_data = np.load('savee.npy') 
tess_data = np.load('tess.npy')
crema_data = np.load('crema.npy')

# ravdess_data.shape, savee_data.shape, tess_data.shape
all_data = np.vstack((ravdess_data, savee_data, tess_data, crema_data))
df = pd.DataFrame(all_data, columns=['label', 'gender', 'pathname', 'filename'])
df.head(), df.shape

# save all data into npy file
np.save("all_datasets.npy", df)

In [3]:
import matplotlib.pyplot as plt
def plot_spec(Y_scale, sr, hop_size, y_axis):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y_scale, sr=sr, hop_length=hop_size, x_axis='time', y_axis= y_axis)
    plt.colorbar(format='%+2.0f dB')

In [9]:
# load wav files using the 'pathname' and 'filename' columns of all_data with librosa
import librosa
import librosa.display
 # import ipy previewer
import IPython.display as ipd

frame_size = 4096 # in samples
hop_size = 1024 # in samples
temporal_chunk_size = 30 # number of temporal bins per sample

# total temporal bins is total_samples/hop_size 


for i in range(2):
    pathname = df['pathname'][i]
    filename = df['filename'][i]

    wav, sr = librosa.load(pwd+ pathname + filename)
    trimmed_wav, _ = librosa.effects.trim(wav, top_db=45)

    if sr != 22050:
        raise ValueError("Sample rate is not 22050Hz")

    S_audio = librosa.stft(trimmed_wav, n_fft=frame_size, hop_length=hop_size)
    y_audio = np.abs(S_audio)
    print(y_audio.shape)


    # split y into chunks of size temporal_chunk_size.
    _y =  (y_audio).T
    split_indices = [(i, len(_y) - 30)[ int(i + 30 >= len(_y))]for i in range(0,len(_y), temporal_chunk_size)]
    raw_chunks = np.vstack([_y[i:i+30].T for i in  split_indices])
    

    # plot a single window from the STFT using librosa 
    # Y_scale = np.abs(s_scale)**2
    # plot_spec(Y_scale=librosa.power_to_db(Y_scale), sr=sr, hop_size=hop_size, y_axis='log')
    
    # extract the mel spectrogram
    mel_spec = librosa.feature.melspectrogram(y=trimmed_wav, sr=sr, n_fft=frame_size, hop_length=hop_size, n_mels=18)
    
    log_mel_spec = librosa.power_to_db(mel_spec)

    librosa.display.specshow(log_mel_spec, sr=sr, hop_length=hop_size, x_axis='time', y_axis='mel')
    plt.show()

    # display both wav and trim_wav
    librosa.display.waveshow(trimmed_wav, sr=sr)
    plt.show()
    # librosa.display.waveshow(wav, sr=sr)
    # plt.show()


# ipd.Audio(filename=pwd+ pathname + filename)

    


(2049, 35)
